In [1]:
# Refactorise apl pour toutes les professions
# Créer le schéma apl et mettre toutes les stats dedans
# Faire la table de correspondance specialite PSP_SPE_SNDS
# tarif_stats ne fonctionne pas
# Faire sur damir_acte_stats : ROW GROUP BY PSP_SPE_SNDS
# Faire damir_age_stage : ROW GROUP BY AGE_BEN_SNDS, PSP_SPE_SNDS

In [2]:
import pandas as pd
import numpy as np
import config
from sqlalchemy import text
print(config.version)
print(config.connection_string)
pd.set_option('display.max_columns', None)

3.2.0.Dev
postgresql://postgres:sa@localhost/icip


In [3]:
specialite=10
year=21
dept=6
source="PS"
min=30
print(year, specialite, source)

21 10 PS


In [4]:
sources: dict[str, str] = {
    "PS": f"""
select ps.id, ps.nom, ps.prenom, an.dept_id, an.id adresse_norm_id, i.id iris, an.lon, an.lat from ps
join tarif t on t.ps_id = ps.id
join tarif_date_source tds on tds.tarif_id=t.id 
join cabinet c on t.cabinet_id=c.id
join adresse_raw ar on c.adresse_raw_id=ar.id
join adresse_norm an on ar.adresse_norm_id=an.id
join specialite_profession sp on sp.profession_id=t.profession_id
join iris.iris i on i.code=an.iris
where sp.specialite_id={specialite}
and tds.date_source_id >= {year}00 and  tds.date_source_id < {year+1}00
and ar.dept_id={dept}
group by ps.id, c.id, an.id, i.id
""",
    "PA": f"""
select pa.id, pa.nom, pa.prenom, an.dept_id, an.id adresse_norm_id, i.id iris, an.lon, an.lat from personne_activite pa
join personne_activite_diplome pad on pad.personne_activite_id=pa.id
join diplome d on pad.diplome_id=d.id
join profession_diplome pd on pd.diplome_id=d.id
join profession p on pd.profession_id=p.id
join specialite_profession sp on sp.profession_id=p.id
join pa_adresse_norm_date_source pands ON pands.personne_activite_id = pa.id
join adresse_norm an on pands.adresse_norm_id=an.id
join iris.iris i on i.code=an.iris
where sp.specialite_id={specialite}
and pands.date_source_id/100={year}
and an.dept_id={dept}
group by pa.id, an.id, i.id
""",
    "RPPS": f"""
select personne.id, personne.nom, personne.prenom, an.dept_id, an.id adresse_norm_id, i.id iris, an.lon, an.lat from personne
join diplome_obtenu dio on dio.personne_id = personne.id
join diplome d on d.id=dio.diplome_id
join profession_diplome pd on pd.diplome_id=d.id
join profession p on pd.profession_id=p.id
join specialite_profession sp on sp.profession_id=p.id
join coord c on c.personne_id=personne.id
join adresse_norm an on c.adresse_norm_id=an.id
join iris.iris i on i.code=an.iris
where sp.specialite_id={specialite}
and personne.date_maj > '20{year}-01-01' and date_effet < '20{year}-01-01'
and an.dept_id={dept}
group by personne.id, an.id, i.id
"""}

sql = sources[source]
print(sql)


select ps.id, ps.nom, ps.prenom, an.dept_id, an.id adresse_norm_id, i.id iris, an.lon, an.lat from ps
join tarif t on t.ps_id = ps.id
join tarif_date_source tds on tds.tarif_id=t.id 
join cabinet c on t.cabinet_id=c.id
join adresse_raw ar on c.adresse_raw_id=ar.id
join adresse_norm an on ar.adresse_norm_id=an.id
join specialite_profession sp on sp.profession_id=t.profession_id
join iris.iris i on i.code=an.iris
where sp.specialite_id=10
and tds.date_source_id >= 2100 and  tds.date_source_id < 2200
and ar.dept_id=6
group by ps.id, c.id, an.id, i.id



In [5]:
df = pd.read_sql(sql, config.connection_string)
df

,id,nom,prenom,dept_id,adresse_norm_id,iris,lon,lat
0,16126,LEONARDI,DOMINIQUE,6,1415,60881903,7.255881,43.709653
1,16130,BENSUSSAN,JEAN,6,92990,60882801,7.227639,43.676026
2,16131,LARA,JEANNINE,6,92963,60880202,7.282865,43.695573
3,16132,GUEGAN,JEAN CLAUDE,6,1425,60270102,7.145627,43.657352
4,16148,CROQUET,PATRICE,6,1433,60690109,6.922279,43.651604
...,...,...,...,...,...,...,...,...
1280,155689,SAROUX,DELPHINE,6,2305,60882401,7.258125,43.695605
1281,155690,PERADOTTO,CYRIL,6,2866,60290121,6.988390,43.550361
1282,155692,DONADIO,MANON,6,77494,61490102,7.313438,43.743446
1283,155827,BOUCHET BADEL,MARIE,6,2328,60690102,6.914032,43.646318


In [6]:
nb_ps = df["id"].nunique()
unique_ps = df.groupby(["id", "lon", "lat"])
print(f"Nb PS {nb_ps}")
print(f"Nb unique PS {len(unique_ps)}")

Nb PS 1203
Nb unique PS 1273


In [7]:
df["key"] = df["id"].astype(str) + "_" + df["lat"].astype(str) + "_" + df["lon"].astype(str)
df["nb_cabinet"] = df.groupby("id")["key"].transform("nunique")
df["nb_cabinet"].value_counts(normalize=True)

nb_cabinet
1    0.893385
2    0.094163
3    0.009339
4    0.003113
Name: proportion, dtype: float64

In [8]:
df["weight"] = 1 / df["nb_cabinet"]
df["nb_per_iris"] = df.groupby("iris")["weight"].transform("sum")
df.head(10)

,id,nom,prenom,dept_id,adresse_norm_id,iris,lon,lat,key,nb_cabinet,weight,nb_per_iris
0,16126,LEONARDI,DOMINIQUE,6,1415,60881903,7.255881,43.709653,16126_43.709653_7.255881,1,1.0,5.000000
1,16130,BENSUSSAN,JEAN,6,92990,60882801,7.227639,43.676026,16130_43.676026_7.227639,1,1.0,8.000000
2,16131,LARA,JEANNINE,6,92963,60880202,7.282865,43.695573,16131_43.695573_7.282865,1,1.0,3.000000
3,16132,GUEGAN,JEAN CLAUDE,6,1425,60270102,7.145627,43.657352,16132_43.657352_7.145627,1,1.0,5.000000
4,16148,CROQUET,PATRICE,6,1433,60690109,6.922279,43.651604,16148_43.651604_6.922279,1,1.0,10.500000
5,16151,DESGATS,MARC,6,1436,60300107,7.005844,43.579045,16151_43.579045_7.005844,1,1.0,5.000000
6,16153,DUBOIS,MICHEL,6,1438,61360000,7.449651,43.877046,16153_43.877046_7.449651,1,1.0,4.000000
7,16159,GIDEL,PIERRE JEAN,6,1466,60882601,7.242220,43.689322,16159_43.689322_7.24222,1,1.0,11.666667
8,16160,MASCHINO,JEAN MARC,6,1467,60290102,7.037066,43.546353,16160_43.546353_7.037066,1,1.0,3.000000
9,16161,SARDOU,YVES,6,1468,60830205,7.492397,43.791639,16161_43.791639_7.492397,1,1.0,1.500000


In [9]:
df = df.sort_values(by='iris')
df = df.drop_duplicates(subset=['iris', 'nb_per_iris'])
df


,id,nom,prenom,dept_id,adresse_norm_id,iris,lon,lat,key,nb_cabinet,weight,nb_per_iris
123,16425,DAEMERS,YVES,6,92989,60040102,7.072160,43.596934,16425_43.596934_7.07216,1,1.0,2.000000
904,113924,DIAMANT,GUY,6,77467,60040103,7.137517,43.567521,113924_43.567521_7.137517,1,1.0,1.000000
109,16404,MANDRILE,DIDIER,6,1651,60040104,7.092388,43.604533,16404_43.604533_7.092388,1,1.0,1.000000
31,16222,NICASTRO,SERGE,6,92975,60040105,7.117538,43.588105,16222_43.588105_7.117538,1,1.0,2.000000
1179,144024,FRICKEY,SARAH,6,2734,60040106,7.112369,43.606074,144024_43.606074_7.112369,1,1.0,8.666667
...,...,...,...,...,...,...,...,...,...,...,...,...
607,17388,ROUSSEAU,FRANCOISE,6,2635,61610103,7.123625,43.658047,17388_43.658047_7.123625,1,1.0,1.000000
1069,132333,AGOSTINI,PIERRE,6,81986,61610104,7.141591,43.641591,132333_43.641591_7.141591,2,0.5,3.000000
742,99028,BENASSAYAG,JEAN JACQUES,6,68080,61610105,7.140329,43.639824,99028_43.639824_7.140329,1,1.0,5.000000
301,16814,DI VINCENZO,DOMINIQUE,6,273951,61620000,7.612673,44.063665,16814_44.063665_7.612673,2,0.5,0.500000


In [10]:
def get_iris_matrix():
    sql = f"""
(select iris.id "iris1", iris.id "iris2", 0 "km", 0 "time", 0 "time_hp" from iris.iris
where id / 10000000 = {dept}
union
select iris_id_from as "iris1", iris_id_to "iris2", route_km "km", route_min "time", route_hp_min "time_hp" from iris.iris_matrix
where (iris_id_from/10000000={dept} or iris_id_to/10000000={dept})
and route_min <= {min}) order by "iris1", "iris2"
"""
    print(sql)
    return pd.read_sql(sql, config.connection_string)

# iris_matrix = get_iris_matrix()
iris_matrix = pd.read_csv("od_06_sql_rename.csv")
iris_matrix["iris"] = iris_matrix["iris2"].astype("int64")
iris_matrix
    

,iris1,iris2,km,time,iris
0,40080000,60530000,17,22,60530000
1,40080000,60990000,21,24,60990000
2,40390000,61540000,24,30,61540000
3,40420000,60530000,11,19,60530000
4,40420000,60710000,22,30,60710000
...,...,...,...,...,...
117764,831380000,60850103,34,29,60850103
117765,831380000,60950101,18,19,60950101
117766,831380000,60950102,19,22,60950102
117767,831380000,60950103,18,20,60950103


In [11]:
def get_pop_iris():
    sql = f"""
select i.id iris, pi.iris iris_string, c.code code_commune, i.type type_iris, pi.p20_pop0002, pi.p20_pop0305, pi.p20_pop0610, pi.p20_pop1117, pi.p20_pop1824, pi.p20_pop2539, pi.p20_pop4054, pi.p20_pop5564, pi.p20_pop6579, pi.p20_pop80p
from iris.pop_iris pi
join iris.iris i on  pi.iris=i.code
join iris.commune c on i.commune_id=c.id
where com like '{dept:02}%'"""
    print(sql)
    return pd.read_sql(text(sql), config.connection_string)

pop_iris = get_pop_iris()
pop_iris


select i.id iris, pi.iris iris_string, c.code code_commune, i.type type_iris, pi.p20_pop0002, pi.p20_pop0305, pi.p20_pop0610, pi.p20_pop1117, pi.p20_pop1824, pi.p20_pop2539, pi.p20_pop4054, pi.p20_pop5564, pi.p20_pop6579, pi.p20_pop80p
from iris.pop_iris pi
join iris.iris i on  pi.iris=i.code
join iris.commune c on i.commune_id=c.id
where com like '06%'


,iris,iris_string,code_commune,type_iris,p20_pop0002,p20_pop0305,p20_pop0610,p20_pop1117,p20_pop1824,p20_pop2539,p20_pop4054,p20_pop5564,p20_pop6579,p20_pop80p
0,60120105,060120105,06012,A,87.418550,60.438694,193.693660,273.95730,223.035700,676.78790,830.08154,389.410980,363.85060,233.009220
1,60500000,060500000,06050,C,14.457494,14.409695,41.336414,48.39683,17.661858,72.61527,146.12288,82.679535,64.89807,26.421946
2,60720000,060720000,06072,C,0.000000,7.000000,5.000000,5.00000,4.000000,22.00000,28.00000,28.000000,33.00000,15.000000
3,61050101,061050101,06105,A,90.654580,136.365310,244.816740,402.70312,211.586750,430.94614,944.59796,513.959000,591.59607,209.481000
4,60150000,060150000,06015,C,36.382305,39.335630,53.229595,83.62182,64.078064,198.46510,268.38065,226.838270,194.07529,71.593270
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,60883403,060883403,06088,A,71.638420,55.635365,104.755210,138.66922,184.708200,294.65110,287.00873,264.283000,267.70267,213.472820
496,60883501,060883501,06088,A,63.991177,86.144790,123.785240,215.96701,182.714100,349.32108,586.11273,378.182900,378.49860,152.377090
497,60883601,060883601,06088,A,100.090430,73.343470,134.026950,215.99605,173.736830,455.11038,490.30264,414.152740,495.68090,194.919170
498,60883602,060883602,06088,A,64.699010,79.082280,140.378510,183.99406,110.133000,398.01132,462.45010,307.656770,423.24580,82.143060


In [12]:
apl = iris_matrix.merge(pop_iris, on="iris", how="left", suffixes=('', ''))
apl

,iris1,iris2,km,time,iris,iris_string,code_commune,type_iris,p20_pop0002,p20_pop0305,p20_pop0610,p20_pop1117,p20_pop1824,p20_pop2539,p20_pop4054,p20_pop5564,p20_pop6579,p20_pop80p
0,40080000,60530000,17,22,60530000,060530000,06053,C,3.000000,3.000000,3.000000,15.00000,3.000000,19.00000,22.00000,20.000000,39.000000,14.00000
1,40080000,60990000,21,24,60990000,060990000,06099,C,45.961033,60.562077,118.186980,174.26068,100.920586,288.92783,375.23218,213.348510,308.132720,140.46739
2,40390000,61540000,24,30,61540000,061540000,06154,C,14.443875,11.072821,23.354239,41.09353,36.612030,60.12047,88.63108,67.417816,107.328766,22.92537
3,40420000,60530000,11,19,60530000,060530000,06053,C,3.000000,3.000000,3.000000,15.00000,3.000000,19.00000,22.00000,20.000000,39.000000,14.00000
4,40420000,60710000,22,30,60710000,060710000,06071,C,13.000000,11.000000,30.000000,25.00000,21.000000,92.00000,105.00000,99.000000,131.000000,58.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117764,831380000,60850103,34,29,60850103,060850103,06085,A,71.068900,71.788710,146.845930,194.39722,148.091400,351.81476,491.94345,358.296630,519.871950,236.63065
117765,831380000,60950101,18,19,60950101,060950101,06095,A,78.000000,88.000000,211.000000,256.00000,155.000000,476.00000,682.00000,415.000000,583.000000,261.00000
117766,831380000,60950102,19,22,60950102,060950102,06095,A,68.000000,78.000000,156.000000,226.00000,145.000000,400.00000,527.00000,351.000000,423.000000,175.00000
117767,831380000,60950103,18,20,60950103,060950103,06095,A,47.000000,69.000000,126.000000,225.00000,103.000000,259.00000,536.00000,385.000000,523.000000,184.00000


In [13]:
accessibilite_fn = lambda x: np.exp(-0.12 * x)

apl["accessibilite_weigth"] = accessibilite_fn(apl["time"])
apl.head(10)

,iris1,iris2,km,time,iris,iris_string,code_commune,type_iris,p20_pop0002,p20_pop0305,p20_pop0610,p20_pop1117,p20_pop1824,p20_pop2539,p20_pop4054,p20_pop5564,p20_pop6579,p20_pop80p,accessibilite_weigth
0,40080000,60530000,17,22,60530000,060530000,06053,C,3.000000,3.000000,3.000000,15.000000,3.000000,19.000000,22.000000,20.000000,39.000000,14.000000,0.071361
1,40080000,60990000,21,24,60990000,060990000,06099,C,45.961033,60.562077,118.186980,174.260680,100.920586,288.927830,375.232180,213.348510,308.132720,140.467390,0.056135
2,40390000,61540000,24,30,61540000,061540000,06154,C,14.443875,11.072821,23.354239,41.093530,36.612030,60.120470,88.631080,67.417816,107.328766,22.925370,0.027324
3,40420000,60530000,11,19,60530000,060530000,06053,C,3.000000,3.000000,3.000000,15.000000,3.000000,19.000000,22.000000,20.000000,39.000000,14.000000,0.102284
4,40420000,60710000,22,30,60710000,060710000,06071,C,13.000000,11.000000,30.000000,25.000000,21.000000,92.000000,105.000000,99.000000,131.000000,58.000000,0.027324
5,40420000,60990000,17,24,60990000,060990000,06099,C,45.961033,60.562077,118.186980,174.260680,100.920586,288.927830,375.232180,213.348510,308.132720,140.467390,0.056135
6,40420000,61240000,15,30,61240000,061240000,06124,C,0.941176,0.000000,1.882353,0.941176,0.941176,2.823529,10.352942,6.588235,19.764706,3.764706,0.027324
7,40430000,60020000,15,20,60020000,060020000,06002,C,0.962963,0.000000,0.962963,0.000000,4.814815,2.888889,12.518518,12.518518,14.444445,2.888889,0.090718
8,40430000,60240000,8,11,60240000,060240000,06024,C,4.785714,5.982143,4.785714,17.946428,3.589286,21.535715,39.482143,22.732143,61.017857,19.142857,0.267135
9,40430000,60450000,18,23,60450000,060450000,06045,C,0.000000,0.000000,2.730769,6.371795,1.820513,2.730769,17.294872,16.384615,19.115385,4.551282,0.063292


In [14]:
cols = [col for col in apl.columns if "pop" in col]
cols

['p20_pop0002',
 'p20_pop0305',
 'p20_pop0610',
 'p20_pop1117',
 'p20_pop1824',
 'p20_pop2539',
 'p20_pop4054',
 'p20_pop5564',
 'p20_pop6579',
 'p20_pop80p']

In [16]:
sql = f"""
select o.* from apl.overrepresentation o
where o.year={year} and o.psp_spe_snds=1
"""
over = pd.read_sql(sql, config.connection_string)
# todo ajouter la colonne psp_spe_snds dans specialite
over

,year,psp_spe_snds,p0002,p0305,p0610,p1117,p1824,p2539,p4054,p5564,p6579,p80p
0,21,1,1.143012,0.52797,0.398005,0.421737,0.712587,0.925556,0.879935,1.076956,1.525866,2.40061


In [17]:
weights = over.values[0,2:]
weights

array([1.14301231, 0.52796983, 0.39800467, 0.42173695, 0.71258726,
       0.92555555, 0.87993485, 1.07695618, 1.52586644, 2.40061021])

In [18]:
apl["pop_gp"] = sum(w * apl[c] for w, c in zip(weights, cols))
apl.sort_values(by='iris2')
apl.head(5)

    

,iris1,iris2,km,time,iris,iris_string,code_commune,type_iris,p20_pop0002,p20_pop0305,p20_pop0610,p20_pop1117,p20_pop1824,p20_pop2539,p20_pop4054,p20_pop5564,p20_pop6579,p20_pop80p,accessibilite_weigth,pop_gp
0,40080000,60530000,17,22,60530000,060530000,06053,C,3.000000,3.000000,3.000000,15.00000,3.000000,19.00000,22.00000,20.000000,39.000000,14.00000,0.071361,166.271356
1,40080000,60990000,21,24,60990000,060990000,06099,C,45.961033,60.562077,118.186980,174.26068,100.920586,288.92783,375.23218,213.348510,308.132720,140.46739,0.056135,1911.697288
2,40390000,61540000,24,30,61540000,061540000,06154,C,14.443875,11.072821,23.354239,41.09353,36.612030,60.12047,88.63108,67.417816,107.328766,22.92537,0.027324,500.115348
3,40420000,60530000,11,19,60530000,060530000,06053,C,3.000000,3.000000,3.000000,15.00000,3.000000,19.00000,22.00000,20.000000,39.000000,14.00000,0.102284,166.271356
4,40420000,60710000,22,30,60710000,060710000,06071,C,13.000000,11.000000,30.000000,25.00000,21.000000,92.00000,105.00000,99.000000,131.000000,58.00000,0.027324,681.401551


In [19]:
apl = apl.drop(["iris", "iris_string", "code_commune"], axis=1)
apl = apl.drop(cols, axis=1)
apl.head(5)

,iris1,iris2,km,time,type_iris,accessibilite_weigth,pop_gp
0,40080000,60530000,17,22,C,0.071361,166.271356
1,40080000,60990000,21,24,C,0.056135,1911.697288
2,40390000,61540000,24,30,C,0.027324,500.115348
3,40420000,60530000,11,19,C,0.102284,166.271356
4,40420000,60710000,22,30,C,0.027324,681.401551


In [21]:
apl["iris"] = apl["iris1"]
apl.sort_values(by='iris2')
apl = apl.merge(df, on="iris", how="left", suffixes=('', ''))
apl

,iris1,iris2,km,time,type_iris,accessibilite_weigth,pop_gp,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,key,nb_cabinet,weight,nb_per_iris
0,40080000,60530000,17,22,C,0.071361,166.271356,40080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,40080000,60990000,21,24,C,0.056135,1911.697288,40080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,40390000,61540000,24,30,C,0.027324,500.115348,40390000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,40420000,60530000,11,19,C,0.102284,166.271356,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,40420000,60710000,22,30,C,0.027324,681.401551,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117764,831380000,60850103,34,29,A,0.030807,2870.777975,831380000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117765,831380000,60950101,18,19,A,0.102284,3441.767196,831380000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117766,831380000,60950102,19,22,A,0.071361,2657.140709,831380000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117767,831380000,60950103,18,20,A,0.090718,2674.319908,831380000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
apl["nb_per_iris"] = apl["nb_per_iris"].fillna(0)
apl = apl.sort_values(by=['iris1', "iris2"])
apl["nb_per_iris"].unique()

array([ 0.        ,  2.        ,  1.        ,  8.66666667,  4.        ,
        1.5       , 13.        ,  3.        ,  0.5       , 23.        ,
       11.5       ,  5.        ,  6.        , 10.        ,  7.        ,
        4.5       , 11.        , 14.        ,  7.5       , 17.        ,
        3.5       ,  7.16666667,  2.5       ,  6.5       , 10.5       ,
        3.16666667,  1.25      ,  8.        , 13.5       , 12.5       ,
       25.66666667,  8.5       ,  9.        , 11.66666667,  7.25      ,
        5.33333333,  5.58333333,  9.5       ])

In [35]:
apl = apl.sort_values(by='iris1')
apl["wpop"] = apl["accessibilite_weigth"] * apl["pop_gp"]
apl["swpop"] = apl.groupby("iris1")["wpop"].transform("sum")
apl["R"] = apl["nb_per_iris"] / (apl["swpop"] / 100000)
apl.head(5)
# apl["R"].unique()

,iris1,iris2,km,time,type_iris,accessibilite_weigth,pop_gp,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,key,nb_cabinet,weight,nb_per_iris,wpop,swpop,R
0,40080000,60530000,17,22,C,0.071361,166.271356,40080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,11.865335,119.178009,0.0
1,40080000,60990000,21,24,C,0.056135,1911.697288,40080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,107.312674,119.178009,0.0
2,40390000,61540000,24,30,C,0.027324,500.115348,40390000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,13.665013,13.665013,0.0
3,40420000,60530000,11,19,C,0.102284,166.271356,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,17.006934,144.602239,0.0
4,40420000,60710000,22,30,C,0.027324,681.401551,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,18.618427,144.602239,0.0


In [37]:
rgp = apl[apl["iris1"] == apl["iris2"]]
rgp = rgp[["iris1", "type_iris", "pop_gp", "nb_per_iris", "R"]].copy()
rgp

,iris1,type_iris,pop_gp,nb_per_iris,R
76,60010000,C,99.689889,0.0,0.000000
87,60020000,C,61.061511,0.0,0.000000
98,60030000,C,626.690248,0.0,0.000000
112,60040101,C,1645.886590,0.0,0.000000
477,60040102,A,2459.845445,2.0,1.325077
...,...,...,...,...,...
115797,61610105,A,3676.152857,5.0,2.461364
116147,61610106,A,3787.415420,0.0,0.000000
116501,61610107,A,2646.705909,0.0,0.000000
116506,61620000,C,786.184842,0.5,24.981019
